<a href="https://colab.research.google.com/github/Dvk2002/NLP/blob/main/HW7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
cd /gdrive/My Drive/Colab_Notebooks/NLP/L7

/gdrive/My Drive/Colab_Notebooks/NLP/L7


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [5]:
data = pd.read_excel('отзывы за лето.xls')

In [ ]:
data

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1
...,...,...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,ну и шляпас роот право бесполезный прогаразраб...,0
20655,5,Ок,2017-06-01,около,1
20656,4,Доволен,2017-06-01,довольный,1
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,песопаснастя рута ни нужын,0


In [6]:
data.Rating.value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [7]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 8.2 MB/s 


In [8]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=57474bac0f2ac396e4b18a7e64b741dbc11679ab51194e753b3c91d4fd294e68
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
pip install gensim

In [9]:
pip install gensim --upgrade

     |████████████████████████████████| 23.9 MB 93 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [10]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
emb_mod=KeyedVectors.load('213/model.model' )

In [12]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [13]:
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import re

exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

In [ ]:
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [58]:

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU,Flatten
from tensorflow.keras.layers import LSTM, Dropout, Input, Bidirectional,Reshape, Activation, BatchNormalization, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [15]:
from sklearn import model_selection, preprocessing

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data.text, data.target)

In [16]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

In [17]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [79]:

vocab_size = 30000
seq_len = 30

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 5),
    output_sequence_length=seq_len
    )

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [80]:
weights=emb_mod[vectorize_layer.get_vocabulary()]

Загружаем веса и не обучаем их

In [81]:
model = tf.keras.Sequential([
  vectorize_layer,
  Embedding(len(vectorize_layer.get_vocabulary()), 300,weights=[weights],trainable=False, mask_zero=True),
  Conv1D(512, 3),
  Activation("relu"),
  GlobalMaxPooling1D(),
  # Dropout(0.2),
  Dense(300, activation='relu'),
  Dropout(0.2),
  Dense(50, activation='relu'),
  Dropout(0.2),
  Dense(10, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='softmax')
  ])

In [82]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
8/8 [==============================] - 20s 2s/step - loss: 0.4296 - accuracy: 0.8471 - val_loss: 0.2566 - val_accuracy: 0.8459
Epoch 2/10
8/8 [==============================] - 19s 2s/step - loss: 0.2335 - accuracy: 0.8501 - val_loss: 0.2407 - val_accuracy: 0.8961
Epoch 3/10
8/8 [==============================] - 19s 2s/step - loss: 0.2136 - accuracy: 0.8821 - val_loss: 0.2225 - val_accuracy: 0.9084
Epoch 4/10
8/8 [==============================] - 19s 2s/step - loss: 0.1970 - accuracy: 0.9068 - val_loss: 0.2124 - val_accuracy: 0.9137
Epoch 5/10
8/8 [==============================] - 19s 2s/step - loss: 0.1830 - accuracy: 0.9151 - val_loss: 0.2060 - val_accuracy: 0.9176
Epoch 6/10
8/8 [==============================] - 19s 2s/step - loss: 0.1691 - accuracy: 0.9251 - val_loss: 0.2061 - val_accuracy: 0.9180
Epoch 7/10
8/8 [==============================] - 19s 2s/step - loss: 0.1580 - accuracy: 0.9305 - val_loss: 0.2009 - val_accuracy: 0.9236
Epoch 8/10
8/8 [==================

Загружаем веса и обучаем их

In [83]:
model = tf.keras.Sequential()
model.add(vectorize_layer)
model.add(Embedding(len(vectorize_layer.get_vocabulary()), 300,weights=[weights],trainable=True, mask_zero=True))
model.add(Conv1D(512, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [84]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
8/8 [==============================] - 39s 5s/step - loss: 0.4407 - accuracy: 0.8422 - val_loss: 0.2393 - val_accuracy: 0.8945
Epoch 2/10
8/8 [==============================] - 39s 5s/step - loss: 0.2201 - accuracy: 0.8957 - val_loss: 0.2192 - val_accuracy: 0.9012
Epoch 3/10
8/8 [==============================] - 38s 5s/step - loss: 0.1928 - accuracy: 0.9137 - val_loss: 0.2005 - val_accuracy: 0.9174
Epoch 4/10
8/8 [==============================] - 38s 5s/step - loss: 0.1672 - accuracy: 0.9299 - val_loss: 0.1917 - val_accuracy: 0.9228
Epoch 5/10
8/8 [==============================] - 38s 5s/step - loss: 0.1440 - accuracy: 0.9408 - val_loss: 0.1896 - val_accuracy: 0.9251
Epoch 6/10
8/8 [==============================] - 38s 5s/step - loss: 0.1244 - accuracy: 0.9511 - val_loss: 0.1926 - val_accuracy: 0.9273
Epoch 7/10
8/8 [==============================] - 38s 5s/step - loss: 0.1070 - accuracy: 0.9621 - val_loss: 0.1863 - val_accuracy: 0.9273
Epoch 8/10
8/8 [==================

Обучаем веса

In [73]:
class model_w(tf.keras.Model):
    def __init__(self):
        super(model_w, self).__init__()
        self.vec = vectorize_layer
        self.emb = Embedding(len(vectorize_layer.get_vocabulary()), 300, mask_zero=True)
        self.act=Activation("relu")
        self.norm=LayerNormalization()
        self.conv=Conv1D(512, 3)
        self.pool = GlobalMaxPooling1D()
        self.drop=Dropout(0.2)     
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(10, activation='relu')
        self.fc3 = Dense(2, activation='softmax')

    def call(self, x):

        x = self.vec(x)
        emb = self.emb(x)
        x= self.conv(emb)
        x=self.act(x)
        pool_x = self.pool(x)       
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(self.drop(fc_x))
        out = self.fc3(self.drop(fc_x))

        return out

In [74]:
model=model_w()
model.compile(optimizer='nadam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
8/8 [==============================] - 40s 5s/step - loss: 0.5374 - accuracy: 0.7638 - val_loss: 0.3680 - val_accuracy: 0.8459
Epoch 2/10
8/8 [==============================] - 39s 5s/step - loss: 0.3551 - accuracy: 0.8471 - val_loss: 0.2703 - val_accuracy: 0.8459
Epoch 3/10
8/8 [==============================] - 38s 5s/step - loss: 0.2632 - accuracy: 0.8485 - val_loss: 0.2141 - val_accuracy: 0.8459
Epoch 4/10
8/8 [==============================] - 39s 5s/step - loss: 0.2032 - accuracy: 0.8963 - val_loss: 0.1800 - val_accuracy: 0.9263
Epoch 5/10
8/8 [==============================] - 38s 5s/step - loss: 0.1525 - accuracy: 0.9454 - val_loss: 0.1850 - val_accuracy: 0.9287
Epoch 6/10
8/8 [==============================] - 38s 5s/step - loss: 0.1256 - accuracy: 0.9547 - val_loss: 0.2102 - val_accuracy: 0.9212
Epoch 7/10
8/8 [==============================] - 38s 5s/step - loss: 0.1018 - accuracy: 0.9655 - val_loss: 0.2004 - val_accuracy: 0.9234
Epoch 8/10
8/8 [==================

Значимой разницы не наблюдается. Модель быстро переобучается, нужно выбирать оптимальное количество эпох или увеличивать регуляризацию.